In [1]:
from typing import Annotated
from typing_extensions import TypedDict
import os
## Working With Tools

from dotenv import load_dotenv
load_dotenv()
from langchain_community.utilities import WikipediaAPIWrapper , DuckDuckGoSearchAPIWrapper, GoogleSearchAPIWrapper
from langchain_community.tools import WikipediaQueryRun , DuckDuckGoSearchResults
from tools import CurrentDateTimeTool, ShippingTimeEstimator, PromoCodeScraper , calculator , get_current_location , scrape_and_crawl
from langchain_core.tools import Tool

In [2]:
from langgraph.store.memory import InMemoryStore

store = InMemoryStore()
store.put(("instructions",), key="agent_instructions", value={"prompt": "Help customer to find it's product and solve it's problem"})

In [3]:
from langgraph.prebuilt import create_react_agent
from langgraph.config import get_store

from langchain_community.utilities import GoogleSearchAPIWrapper , DuckDuckGoSearchAPIWrapper
from langchain_core.tools import Tool

search = GoogleSearchAPIWrapper()



Google_search = Tool(
    name="google_search",
    description="Search Google for recent results.",
    func=search.run,
)


api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=300)
wiki_tool=WikipediaQueryRun(api_wrapper=api_wrapper)

api_wrapper = DuckDuckGoSearchAPIWrapper(region="de-de", time="d", max_results=2)
DuckDuckGo = DuckDuckGoSearchResults(api_wrapper=api_wrapper, source="news")

def prompt(state):
    item = store.get(("instructions",), key="agent_instructions")
    instructions = item.value["prompt"]
    sys_prompt = {"role": "system", "content": f"## Instructions\n\n{instructions}"}
    return [sys_prompt] + state['messages']

agent = create_react_agent("gpt-3.5-turbo", prompt=prompt, tools=[Google_search,DuckDuckGo], store=store)

/var/folders/2v/86sq867s35xc02tyt5c0rwh00000gp/T/ipykernel_2001/1764391507.py:7: LangChainDeprecationWarning: The class `GoogleSearchAPIWrapper` was deprecated in LangChain 0.0.33 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-google-community package and should be used instead. To use it run `pip install -U :class:`~langchain-google-community` and import as `from :class:`~langchain_google_community import GoogleSearchAPIWrapper``.
  search = GoogleSearchAPIWrapper()


In [4]:
result = agent.invoke(
    {"messages": [
        {"role": "user", "content" :"where is dubai?"}]}
)
result['messages'][-1].pretty_print()

================================== Ai Message ==================================

Dubai is located on the Persian Gulf coast of the United Arab Emirates. It shares borders with Abu Dhabi in the south and Sharjah in the northeast.


In [5]:
from langmem import create_prompt_optimizer

optimizer = create_prompt_optimizer("gpt-3.5-turbo")

In [6]:
current_prompt = store.get(("instructions",), key="agent_instructions").value["prompt"]
feedback = {"request": "Always sign off from 'Google Ggent'; Hope you enjoyed web surffing with me"}

optimizer_result = optimizer.invoke({"prompt": current_prompt, "trajectories": [(result["messages"], feedback)]})

In [7]:
print(optimizer_result)

Help customer to find its product and solve its problem. Thank you for reaching out to Google support. If you need any further assistance, please feel free to ask. - Google Agent


In [8]:
store.put(("instructions",), key="agent_instructions", value={"prompt": optimizer_result})

In [9]:
result = agent.invoke(
    {"messages": [
        {"role": "user", "content" :"what famous in dubai?"}]}
)
result['messages'][-1].pretty_print()

================================== Ai Message ==================================

Some famous places in Dubai include:
1. Burj Khalifa
2. Aquaventure Waterpark
3. The Dubai Fountain
4. Burj Al Arab
5. Global Village
6. Dubai Mall
7. Ski Dubai
8. Desert Safari
9. Dubai Garden Glow
10. Palm Jumeirah

These are just a few of the many attractions Dubai has to offer. Let me know if you need more information or recommendations!


In [10]:
result = agent.invoke(
    {"messages": [
        {"role": "user", "content" : "what are the price ticket from india to dubai?"}]}
)
result['messages'][-1].pretty_print()

================================== Ai Message ==================================

The price of a ticket from India to Dubai may vary depending on the airline, the time of booking, and other factors. On average, flight tickets from India to Dubai can cost around ₹10,000. It is recommended to book your flight around 45-60 days in advance for the best deals.


In [11]:
from langgraph.store.memory import InMemoryStore
from langgraph.prebuilt import create_react_agent
from langgraph.config import get_store
from tools import CurrentDateTimeTool, ShippingTimeEstimator, PromoCodeScraper , calculator , get_current_location , scrape_and_crawl

In [12]:
from langchain_community.utilities import WikipediaAPIWrapper , DuckDuckGoSearchAPIWrapper, GoogleSearchAPIWrapper
from langchain_community.tools import WikipediaQueryRun , DuckDuckGoSearchResults
from tools import CurrentDateTimeTool, ShippingTimeEstimator, PromoCodeScraper , calculator , get_current_location , scrape_and_crawl
from langchain_core.tools import Tool

In [13]:
api_wrapper = DuckDuckGoSearchAPIWrapper(region="de-de", time="d", max_results=2)
DuckDuckGo = DuckDuckGoSearchResults(api_wrapper=api_wrapper, source="news")

api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=300)
wiki_tool=WikipediaQueryRun(api_wrapper=api_wrapper)

In [14]:
store = InMemoryStore()

In [15]:
store.put(("instructions",), key="search_agent", value={"prompt": "Search the web for relevant information."})
store.put(("instructions",), key="datetimeloc_agent", value={"prompt": "Retrieve the current date,time and user's location."})


In [16]:

def prompt_search_agent(state):
    item = store.get(("instructions",), key="search_agent")
    instructions = item.value["prompt"]
    sys_prompt = {"role": "system", "content": f"## Instructions\n\n{instructions}"}
    return [sys_prompt] + state['messages']
def prompt_datetimeloc_agent(state):
    item = store.get(("instructions",), key="datetimeloc_agent")
    instructions = item.value["prompt"]
    sys_prompt = {"role": "system", "content": f"## Instructions\n\n{instructions}"}
    return [sys_prompt] + state['messages']

In [34]:
from typing import Annotated

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langgraph.constants import END, START
from typing_extensions import TypedDict
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import ToolNode, tools_condition

In [17]:
search_node = ToolNode(tools=[Google_search,DuckDuckGo])
curr_time_loc_node = ToolNode(tools=[CurrentDateTimeTool,get_current_location])

def call_search(state):
    messages = state["messages"]
    tool_messages = search_node.invoke([messages[-1]])
    return {"messages": [message for message in tool_messages if message.name == "search"]}

def call_search(state):
    messages = state["messages"]
    tool_messages = search_node.invoke([messages[-1]])
    return {"messages": [message for message in tool_messages if message.name == "search"]}


search_agent = create_react_agent(
    "gpt-3.5-turbo", 
    prompt=prompt_search_agent, 
    tools=[Google_search,DuckDuckGo], 
    store=store,
    name="search_agent",
)

datetimeloc_agent = create_react_agent(
    "gpt-3.5-turbo", 
    prompt=prompt_datetimeloc_agent, 
    tools=[CurrentDateTimeTool,get_current_location], 
    store=store,
    name="datetimeloc_agent",
)


In [18]:
from langgraph_supervisor import create_supervisor

# Create supervisor workflow
workflow = create_supervisor(
    [search_agent,datetimeloc_agent],
    model="gpt-3.5-turbo",
    prompt=(
        "You are a team supervisor managing Google and DuckDuckGo assistants to help with correspondance."
    )
)



app = workflow.compile( store=store)

In [19]:
result = app.invoke(
    {"messages": [
        {"role": "user", "content" :"where is jibhi? and how much far it's from me"}]},
)

In [20]:
result["messages"][-1].pretty_print()

================================== Ai Message ==================================
Name: supervisor

Jibhi is located in the Banjar Valley of Himachal Pradesh, India, and is approximately 8 kilometers from Banjar. The distance between you, in Panipat, Haryana, India, and Jibhi would be several hundred kilometers.


In [21]:
from langmem import create_multi_prompt_optimizer

feedback = {"request": "Always sign off from 'Google Ggent'; Hope you enjoyed web surffing with me"}

optimizer = create_multi_prompt_optimizer("gpt-3.5-turbo")

In [22]:
from langmem import Prompt
?Prompt

Init signature: Prompt(self, /, *args, **kwargs)
Docstring:     
TypedDict for structured prompt management and optimization.

Example:
    ```python
    from langmem import Prompt

    prompt = Prompt(
        name="extract_entities",
        prompt="Extract key entities from the text:",
        update_instructions="Make minimal changes, only address where"
        " errors have occurred after reasoning over why they occur.",
        when_to_update="If there seem to be errors in recall of named entities.",
    )
    ```

The name and prompt fields are required. Optional fields control optimization:
- update_instructions: Guidelines for modifying the prompt
- when_to_update: Dependencies between prompts during optimization

Use in the prompt optimizers.
File:           /opt/anaconda3/envs/llm_agent/lib/python3.13/site-packages/langmem/prompts/types.py
Type:           _TypedDictMeta
Subclasses:     

In [26]:
search_prompt = store.get(("instructions",), key="search_agent").value['prompt']
datetimeloc_prompt = store.get(("instructions",), key="datetimeloc_agent").value['prompt']

search_prompt = {
    "name": "search_agent",
    "prompt": search_prompt,
    "when_to_update": "Only if feedback is provided indicating web searching performance needs improved."
}

datetimeloc_prompt = {
    "name": "datetimeloc_agent",
    "prompt": datetimeloc_prompt,
    "when_to_update": "Update if the date/time format needs localization, time zones are incorrect, or response speed is slow. and give the location of the user as possible as can."
}

optimizer_result = optimizer.invoke({"prompts": [search_prompt,datetimeloc_prompt], "trajectories": [(result["messages"], feedback)]})

In [27]:
optimizer_result

[{'name': 'search_agent',
  'prompt': "Enhance the user experience by infusing a more engaging and personal tone in the prompt. Sign-off using 'Google Agent' to improve professionalism and correctness. Google Agent",
  'when_to_update': 'Only if feedback is provided indicating web searching performance needs improved.'},
 {'name': 'datetimeloc_agent',
  'prompt': "Retrieve the current date and time in the format 'YYYY-MM-DD HH:MM:SS' and the user's location using GPS coordinates.",
  'when_to_update': 'Update if the date/time format needs localization, time zones are incorrect, or response speed is slow. and give the location of the user as possible as can.'}]

In [29]:
for i in range(len(optimizer_result)):
    store.put(("instructions",), key=optimizer_result[i]['name'], value={"prompt": optimizer_result[i]['prompt']})

In [32]:
result = app.invoke(
    {"messages": [
        {"role": "user", "content" :"Where is dwarka? and how much it is from me?"}]},
)

ValueError: Found AIMessages with tool_calls that do not have a corresponding ToolMessage. Here are the first few of those tool calls: [{'name': 'transfer_to_datetimeloc_agent', 'args': {}, 'id': 'call_YNKr4JeNIQS3i9eFisqJC2kG', 'type': 'tool_call'}].

Every tool call (LLM requesting to call a tool) in the message history MUST have a corresponding ToolMessage (result of a tool invocation to return to the LLM) - this is required by most LLM providers.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_CHAT_HISTORY

In [31]:
result["messages"][-1].pretty_print()

================================== Ai Message ==================================
Name: supervisor

Dwarka is a historic town located on the western coast of the Okhamandal Peninsula in Gujarat, India. It is known for its significance in Hindu mythology and as a sacred pilgrimage site. If you need more information or assistance, feel free to ask!
